The first step in this project is to create the test points. In theory, this can be done manually, but in order to scale the project, it is best to automatically create all of the test files. In order to run this, there needs to be a template "Test" file for a baseline simulation. This file will be read and the appropriate values will be replaced and copied for many test files. 

Here, the pyDOE2 library is used to create a fractional factorial design. This will assign combinations of test conditions to cover a wide range of conditions while limiting the number of data points.

In [27]:
original = "Test_RVE"
file = open(original + ".vt",'r')
data = file.readlines()
data = [x.strip("\t") for x in data]

import pyDOE2
import numpy as np

#Create factorial design
strains = pyDOE2.gsd([3,3,3],2)
strains = strains + 1
strains = np.insert(strains, 2, 4, axis=1)
strains = np.insert(strains, 4, 4, axis=1)
strains = np.insert(strains, 5, 4, axis=1)

#Time Step array
ts = [0.001, 0.005, 0.01, 0.05, 0.1]

for i in range(len(strains)):
    for j in range(len(ts)):
        f = open(original + "v_" + str(i) + str(j) + ".vt", "w+")
        for k in range(len(data)):
            if k == 6:
                f.write(data[k].replace("0.03", str(ts[j]), 3))
            elif k == 16:
                f.write(data[k].replace("33", str(int(1/ts[j])), 1))
            else:
                f.write(data[k].replace("\t1\t1\t1\t1\t1\t1", "\t" + str(strains[i,0]) + "\t" + str(strains[i,1]) + \
                                        "\t" + str(strains[i,2]) + "\t" + str(strains[i,3]) + \
                                        "\t" + str(strains[i,4]) + "\t" + str(strains[i,5])))
  

In [31]:
test = np.zeros([len(strains)*len(ts), 13])
strains = np.where(strains == 1, 0.05, strains)
strains = np.where(strains == 2, 0.015, strains)
strains = np.where(strains == 3, 0.005, strains)
strains = np.where(strains == 4, 0.0015, strains)
count = 0
for i in range(len(strains)):
    for j in range(len(ts)):
        x = np.append(strains[i], ts[j])
        x = np.append(x, strains[i]*ts[j])
        test[count] = x
        count = count + 1
        
np.savetxt("Test_Cases.csv", test, delimiter=",", header = "Stain 11,Strain 22,Strain 33,Strain 12,Strain 13,Strain 23," + \
           "Time Step,Stain 11 per TS,Strain 22 per TS,Strain 33 per TS,Strain 12 per TS,Strain 13 per TS," + \
           "Strain 23 per TS," + str(len(strains)) + "," + str(len(ts)))